<a href="https://colab.research.google.com/github/ccervantes369/benchmark-function-comparison/blob/main/COMITvsUMDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Librerías

In [ ]:
import pandas as pd
import io
import numpy as np
import math
import networkx as nx
import scipy.stats as stats

### Funciones

In [ ]:
def ackley(x):
    t1 = -20 * np.exp(-0.2 * np.sqrt(np.sum(x**2, axis=1) / x.shape[1]))
    t2 = -np.exp(np.sum(np.cos(2 * np.pi * x), axis=1) / x.shape[1])
    return t1 + t2 + np.e + 20

def griewangk(x):
    sum_term = np.sum(x**2 / 4000, axis=1)
    prod_term = np.prod(np.cos(x / np.sqrt(np.arange(1, x.shape[1] + 1))), axis=1)
    return sum_term - prod_term + 1

def rastrigin(x):
    return 10 * x.shape[1] + np.sum(x**2 - 10 * np.cos(2 * np.pi * x), axis=1)

def rosenbrock(x):
    return np.sum(100 * (x[:, 1:] - x[:, :-1]**2)**2 + (1 - x[:, :-1])**2, axis=1)

def weierstrass(x, a=0.5, b=3, k_max=20):
    k = np.arange(k_max)
    term1 = np.sum([a**k * np.cos(2 * np.pi * b**k * (x + 0.5)) for k in k], axis=0)
    term2 = np.sum([a**k * np.cos(2 * np.pi * b**k * 0.5) for k in k])
    return np.sum(term1, axis=1) - x.shape[1] * term2

def happy_cat(x, alpha=0.125):
    norm_x2 = np.sum(x**2, axis=1)
    term1 = np.abs(norm_x2 - x.shape[1])**(2 * alpha)
    term2 = (0.5 * norm_x2 + np.sum(x, axis=1)) / x.shape[1]
    return term1 + term2 + 0.5


funciones = [ackley, griewangk, rastrigin, rosenbrock, weierstrass, happy_cat]

# Algoritmo UMDA

In [ ]:
def umda(funcion, n, d, iteraciones, repeticiones):
    poblacion = np.random.uniform(-100, 100, size=(n, d))
    mejores_por_repeticion = []

    for rep in range(repeticiones):
        mejor_aptitud_rep = np.inf
        for i in range(iteraciones):
            aptitudes = funcion(poblacion)

            mejor_aptitud_rep = min(mejor_aptitud_rep, np.min(aptitudes))

            m = math.floor(n / 2)
            indices = np.argsort(aptitudes)[:m]
            mejores = poblacion[indices]

            mu = mejores.mean(axis=0)
            sd = mejores.std(axis=0)

            poblacion = np.random.normal(mu, sd, size=(n, d))

        mejores_por_repeticion.append(mejor_aptitud_rep)

    mejores_por_repeticion = np.array(mejores_por_repeticion)
    minimo = np.min(mejores_por_repeticion)
    promedio = np.mean(mejores_por_repeticion)
    maximo = np.max(mejores_por_repeticion)
    desviacion_estandar = np.std(mejores_por_repeticion)
    mediana = np.median(mejores_por_repeticion)

    return (mejores_por_repeticion, minimo, promedio, maximo, desviacion_estandar, mediana)


# COMIT

In [ ]:
def sample_new_population(mejores, n, d):
    epsilon = 1e-8
    mu = mejores.mean(axis=0)
    sd = mejores.std(axis=0)
    corr = np.corrcoef(mejores, rowvar=False)

    G = nx.Graph()
    G.add_nodes_from(range(d))

    i, j = np.triu_indices(d, k=1)
    weights = 0.5 * np.log(1 - np.square(corr[i, j]))
    edges = list(zip(i, j, weights))
    G.add_weighted_edges_from(edges)

    T = nx.minimum_spanning_tree(G)
    root = 0
    arbol_dirigido = nx.bfs_tree(T, root)

    nueva_poblacion = np.zeros((n, d))
    nueva_poblacion[:, root] = np.random.normal(mu[root], sd[root], size=n)

    bfs_edges = list(nx.bfs_edges(arbol_dirigido, root))

    for padre, hijo in bfs_edges:
        rho = corr[padre, hijo]
        ratio_sd = sd[hijo] / sd[padre]
        mu_cond = mu[hijo] + ratio_sd * rho * (nueva_poblacion[:, padre] - mu[padre])
        var_cond = max(0, 1 - rho**2) * sd[hijo] ** 2
        sd_cond = math.sqrt(var_cond)
        nueva_poblacion[:, hijo] = np.random.normal(mu_cond, sd_cond)

    return nueva_poblacion


def comit(funcion, n, d, iteraciones, repeticiones):
    poblacion = np.random.uniform(-100, 100, size=(n, d))
    mejores_por_repeticion = []

    for _ in range(repeticiones):
        mejor_aptitud_rep = np.inf
        poblacion_actual = poblacion.copy()

        for __ in range(iteraciones):
            aptitudes = funcion(poblacion_actual)
            mejor_aptitud_rep = min(mejor_aptitud_rep, np.min(aptitudes))

            m = math.floor(n / 2)
            indices = np.argsort(aptitudes)[:m]
            mejores = poblacion_actual[indices]

            poblacion_actual = sample_new_population(mejores, n, d)

        mejores_por_repeticion.append(mejor_aptitud_rep)

    mejores_por_repeticion = np.array(mejores_por_repeticion)
    minimo = np.min(mejores_por_repeticion)
    promedio = np.mean(mejores_por_repeticion)
    maximo = np.max(mejores_por_repeticion)
    desviacion_estandar = np.std(mejores_por_repeticion)
    mediana = np.median(mejores_por_repeticion)

    return (mejores_por_repeticion, minimo, promedio, maximo, desviacion_estandar, mediana)


# Pruebas de Hipótesis

In [ ]:
# Función de prueba de normalidad
def prueba_normalidad(datos, nombre_metodo):
    stat, p_value = stats.shapiro(datos)
    print(f"Prueba de normalidad ({nombre_metodo}):")
    print(f"Estadístico de la prueba: {stat:.4f} | Valor p: {p_value:.4f}")

    if p_value > 0.05:
        print(f"Los datos de {nombre_metodo} siguen una distribución normal.\n")
        return True
    else:
        print(f"Los datos de {nombre_metodo} NO siguen una distribución normal.\n")
        return False

# Función para la prueba t de Student (prueba de diferencia de medias)
def prueba_diferencia_de_medias(datos_comit, datos_umda):
    stat, p_value = stats.ttest_ind(datos_comit, datos_umda, equal_var=False)
    print(f"\nPrueba de diferencia de medias (t-Student):")
    print(f"Estadístico de la prueba: {stat:.4f} | Valor p: {p_value:.4f}")

    # Si el valor p es menor que 0.05, rechazamos H0
    if p_value < 0.05:
        print("Se rechaza H0: Hay una diferencia significativa entre los promedios de COMIT y UMDA.\n")
    else:
        print("No se rechaza H0: No hay evidencia suficiente para afirmar que hay diferencia significativa.\n")

# Función para la prueba de hipótesis no paramétrica (Mann-Whitney U) con H0: mu1 >= mu2
def prueba_no_parametrica(datos_comit, datos_umda):
    # La prueba 'alternative="less"' verifica H₀: mu1 >= mu2 vs. H0: mu1 < mu2
    stat, p_value = stats.mannwhitneyu(datos_comit, datos_umda, alternative='less')
    print(f"Prueba no paramétrica (Mann-Whitney U):")
    print(f"Estadístico de la prueba: {stat:.4f} | Valor p: {p_value:.4f}")

    # Si el valor p es menor que 0.05, rechazamos H₀
    if p_value < 0.05:
        print("Se rechaza H0: El promedio de COMIT es significativamente menor que el de UMDA.\n")
    else:
        print("No se rechaza H0: No hay evidencia suficiente para afirmar que el promedio de COMIT sea menor que el de UMDA.\n")


# Resultados

In [ ]:
# Parámetros
d_values = [5, 10]  # Lista de valores de d
iteraciones = 100
repeticiones = 30

# Ejecutamos la optimización para cada valor de d
for d in d_values:
    n = 20 * d  # Número de variables basado en d
    print(f"\n=====================\nResultados para d = {d}\n=====================")

    for funcion in funciones:
        # Evaluamos con COMIT
        resultados_comit, minimo_comit, promedio_comit, maximo_comit, desviacion_estandar_comit, mediana_comit = comit(funcion, n, d, iteraciones, repeticiones)

        # Evaluamos con UMDA
        resultados_umda, minimo_umda, promedio_umda, maximo_umda, desviacion_estandar_umda, mediana_umda = umda(funcion, n, d, iteraciones, repeticiones)

        # Resultados para COMIT
        print(f"\nResultados para {funcion.__name__} usando COMIT:")
        print(f"Mínimo: {minimo_comit:.4f} | Promedio: {promedio_comit:.4f} | Máximo: {maximo_comit:.4f}")
        print(f"Desviación estándar: {desviacion_estandar_comit:.4f} | Mediana: {mediana_comit:.4f}")

        # Resultados para UMDA
        print(f"\nResultados para {funcion.__name__} usando UMDA:")
        print(f"Mínimo: {minimo_umda:.4f} | Promedio: {promedio_umda:.4f} | Máximo: {maximo_umda:.4f}")
        print(f"Desviación estándar: {desviacion_estandar_umda:.4f} | Mediana: {mediana_umda:.4f} \n")

        # Realizar prueba de normalidad para COMIT
        normal_comit = prueba_normalidad(resultados_comit, "COMIT")

        # Realizar prueba de normalidad para UMDA
        normal_umda = prueba_normalidad(resultados_umda, "UMDA")

        # Si ambos son normales, realizamos la prueba t de diferencia de medias
        if normal_comit and normal_umda:
            print("\n*** Los datos siguen una distribución normal, realizando prueba t de diferencia de medias ***\n")
            prueba_diferencia_de_medias(resultados_comit, resultados_umda)

        # Si alguno de los dos no es normal, realizamos la prueba no paramétrica
        else:
            print("\n*** Al menos uno de los datos NO sigue una distribución normal, realizando prueba no paramétrica ***\n")
            prueba_no_parametrica(resultados_comit, resultados_umda)


Resultados para d = 5

Resultados para ackley usando COMIT:
Mínimo: 0.0000 | Promedio: 0.0025 | Máximo: 0.0742
Desviación estándar: 0.0133 | Mediana: 0.0000

Resultados para ackley usando UMDA:
Mínimo: 0.0000 | Promedio: 0.0000 | Máximo: 0.0000
Desviación estándar: 0.0000 | Mediana: 0.0000 

Prueba de normalidad (COMIT):
Estadístico de la prueba: 0.1796 | Valor p: 0.0000
Los datos de COMIT NO siguen una distribución normal.

Prueba de normalidad (UMDA):
Estadístico de la prueba: 0.1796 | Valor p: 0.0000
Los datos de UMDA NO siguen una distribución normal.


*** Al menos uno de los datos NO sigue una distribución normal, realizando prueba no paramétrica ***

Prueba no paramétrica (Mann-Whitney U):
Estadístico de la prueba: 873.0000 | Valor p: 1.0000
No se rechaza H0: No hay evidencia suficiente para afirmar que el promedio de COMIT sea menor que el de UMDA.


Resultados para griewangk usando COMIT:
Mínimo: 0.0000 | Promedio: 0.0129 | Máximo: 0.1331
Desviación estándar: 0.0307 | Mediana